<h2><center> Welcome to the TotalEnergies rEVolution Hackathon</h2></center>
<figure>
<!--<img src ="https://drive.google.com/uc?export=view&id=1hSOAfRhJ_jo-MZAjq81VYJu5bZNL7EjD" width = "800" height = '500'/> -->

***Pelogue***
>TotalEnergies Uganda welcomes you to the TotalEnergies rEVolution Hackathon, implemented by Outbox Uganda, to deliver optimal locations of EV charging points in Kampala considering several constraints like traffic, optimum coverage and population density and usage.

> This challenge serves as a qualification for the grand challenge NB: This challenge serves as a qualifier to be selected for the [TotalEnergies rEVolution Hackathon ($10 000 in prizes)](https://zindi.africa/competitions/total-energies-revolution-hackathon-finals) to be hosted virtually on 18-20 September 2023. To qualify for the hackathon, you need to be in a team of 4 members and make a submission to this challenge.

***About the problem***
>TotalEnergies Uganda welcomes you to the TotalEnergies rEVolution Hackathon, implemented by Outbox Uganda, to deliver optimal locations of EV charging points in Kampala considering several constraints like traffic, optimum coverage and population density and usage.

>The lack of optimal locations for electric vehicle (EV) charging points in Kampala presents a significant challenge for the adoption and usage of electric vehicles. Factors such as traffic congestion, optimum coverage, population density, and usage patterns need to be considered to ensure the effective implementation of EV charging infrastructure. Therefore, there is a need to address this problem by identifying and delivering the most suitable locations for EV charging points in Kampala.

>Nairobi is one of the most heavily congested cities in Africa. Each day thousands of Kenyans make the trip into Nairobi from towns such as Kisii, Keroka, and beyond for work, business, or to visit friends and family. The journey can be long, and the final approach into the city can impact the length of the trip significantly depending on traffic. How do traffic patterns influence people's decisions to come into the city by bus and which bus to take? Does knowing the traffic patterns in Nairobi help anticipate the demand for particular routes at particular times?

***Objective of this challenge***
> The primary objective of the hackathon is to develop innovative solutions that can identify the optimal locations for EV charging points in Kampala. Participants will be encouraged to leverage data analytics, machine learning, and other relevant technologies to analyse various constraints, such as traffic patterns, coverage requirements, population density, and EV usage.

>The aim of the competition is to create a predictive model using traffic data provided from Uber Movement and historic bus ticket sales data from Mobiticket to predict the number of tickets that will be sold for buses into Nairobi from cities in "up country" Kenya.

***About the Data***
>The data used to train the model will be historic hourly traffic patterns in Nairobi and historic ticket purchasing data for 14 bus routes into Nairobi from October 2017 to April 2018, and includes the place or origin, the scheduled time of departure, the channel used for the purchase, the type of vehicle, the capacity of the vehicle, and the assigned seat number. Zindi competitors will be allowed to create their own customized traffic datasets using the Uber Movement platform.

***Evaluation metric***
> The Mean Absolute Error will be used to evaluate accuracy of the submitted solutions. So the lower the score the better!

***Relevance of the Challenge***
>This resulting model can be used to anticipate customer demand for certain rides, to manage resources and vehicles more efficiently, to offer promotions and sell other services more effectively, such as micro-insurance, or even improve customer service by being able to send alerts and other useful information to customers

>The solutions to this challenge are the first step towards solving Nairobi's traffic problems. We look forward to taking this journey with you!

### Install the required libraries

In [3]:
!pip install catboost
!pip install google.colab

  Using cached google-colab-1.0.0.tar.gz (72 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Using cached ipykernel-4.6.1-py3-none-any.whl (104 kB)
  Using cached ipython-5.5.0-py3-none-any.whl (758 kB)
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached six-1.12.0-py2.py3-none-any.whl (10 kB)
  Using cached pandas-0.24.2.tar.gz (11.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [15 lines of output]
  C:\Users\michael\AppData\Local\Temp\pip-install-mwb0nqhd\pandas_29ea84b1f7c7445fb58aceeac05bd9c7\setup.py:12: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
    import pkg_resources
  C:\ProgramData\anaconda3\envs\hackthons\Lib\site-packages\setuptools\__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
  !!
  
          ********************************************************************************
          Requirements should be satisfied by a PEP 517 installer.
          If you are using pip, you can try `pip install --use-pep517`.
          ********************************************************************************
  
  !!
    dist.fetch_build_eggs(dist.setup_requires)
  error in pandas setup command: 'install_requires' must b

### Import libraries

In [2]:
import pandas as pd
import random
import os
import numpy as np
%matplotlib inline
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error
from google.colab import drive

ModuleNotFoundError: No module named 'google'

### Seed the random number generator

In [ ]:
# Set seed for reproducibility
SEED = 2023
random.seed(SEED)
np.random.seed(SEED)

### Mount Google Drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')
DATA_PATH = '/content/drive/MyDrive/data-totalenergies/'


In [68]:
# Load training data, reshape, add departure time as an integer number of seconds and add day of week:
df = pd.read_csv(os.path.join(DATA_PATH,'Train.csv'), parse_dates=['travel_date'], dayfirst=False)
train = df.groupby(['ride_id', 'travel_date', 'travel_time', 'travel_from', 'max_capacity']).size().reset_index(name='Count') #sort=False if needed?
train["travel_time"] = train["travel_time"].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
train['day'] = train['travel_date'].dt.dayofweek

C:\Users\michael\AppData\Local\Temp\ipykernel_11424\973339120.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('../TotalEnergies/data/Train.csv', parse_dates=['travel_date'], dayfirst=False)


In [69]:
# The same for the test data
test = pd.read_csv(os.path.join(DATA_PATH,'Test.csv'), parse_dates=['travel_date'], dayfirst=False).drop(['car_type', 'travel_to'], axis=1)
test["travel_time"] = test["travel_time"].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
test['day'] = test['travel_date'].dt.dayofweek

In [70]:
# The sample submission file
sample = pd.read_csv(os.path.join(DATA_PATH,'SampleSubmission.csv'))

In [71]:
# Combine training and test data for now, so that we can add uber movement data all in one go
train['t'] = 0
test['t'] = 1
X = pd.concat([train, test], sort=False)

In [72]:
# Load travel times from Uber movement data ( 3 x 3month periods)
"""
t1 = pd.read_csv('../TotalEnergies/data/Travel_Times_Daily_1.csv',parse_dates=['Date'])
t2 = pd.read_csv('../TotalEnergies/data/Travel_Times_Daily_2.csv',parse_dates=['Date'])
t3 = pd.read_csv('../TotalEnergies/data/Travel_Times_Daily_3.csv',parse_dates=['Date'])
"""

t4 = pd.read_csv(os.path.join(DATA_PATH,'Travel_Times_Awendo.csv'),parse_dates=['Date'])
t5 = pd.read_csv(os.path.join(DATA_PATH,'Travel_Times_HomaBay.csv'),parse_dates=['Date'])
t6 = pd.read_csv(os.path.join(DATA_PATH,'Travel_Times_Kehancha.csv'),parse_dates=['Date'])
t7 = pd.read_csv(os.path.join(DATA_PATH,'Travel_Times_KenduBay.csv'),parse_dates=['Date'])
t8 = pd.read_csv(os.path.join(DATA_PATH,'Travel_Times_Keroka.csv'),parse_dates=['Date'])
t9 = pd.read_csv(os.path.join(DATA_PATH,'Travel_Times_Keumbu.csv'),parse_dates=['Date'])
t10 = pd.read_csv(os.path.join(DATA_PATH,'Travel_Times_Kijauri.csv'),parse_dates=['Date'])
t11 = pd.read_csv(os.path.join(DATA_PATH,'Travel_Times_Kisii.csv'),parse_dates=['Date'])
t12 = pd.read_csv(os.path.join(DATA_PATH,'Travel_Times_Mbita.csv'),parse_dates=['Date'])
t13 = pd.read_csv(os.path.join(DATA_PATH,'Travel_Times_Migori.csv'),parse_dates=['Date'])
t14 = pd.read_csv(os.path.join(DATA_PATH,'Travel_Times_Ndhiwa.csv'),parse_dates=['Date'])
t15 = pd.read_csv(os.path.join(DATA_PATH,'Travel_Times_Nyachenge.csv'),parse_dates=['Date'])
t16 = pd.read_csv(os.path.join(DATA_PATH,'Travel_Times_Oyugis.csv'),parse_dates=['Date'])
t17 = pd.read_csv(os.path.join(DATA_PATH,'Travel_Times_Rodi.csv'),parse_dates=['Date'])
t18 = pd.read_csv(os.path.join(DATA_PATH,'Travel_Times_Rongo.csv'),parse_dates=['Date'])
t19 = pd.read_csv(os.path.join(DATA_PATH,'Travel_Times_Sirare.csv'),parse_dates=['Date'])
t20 = pd.read_csv(os.path.join(DATA_PATH,'Travel_Times_Sori.csv'),parse_dates=['Date'])

travel_times = pd.concat([t4, t5, t6, t7, t8, t9, t10, t11, t12, t13, t14, t15, t16, t17, t18, t19, t20], ignore_index=True)
travel_times = travel_times.fillna(method='ffill')[['Daily Mean Travel Time (Seconds)', 'Date']]
travel_times['Date'] = pd.to_datetime(travel_times['Date'])
travel_times.head(1)

,Daily Mean Travel Time (Seconds),Date
0,196.0,2017-10-01


In [73]:
# Merge with our contest data
X['Date'] = X['travel_date']
X.set_index('travel_date', inplace=True)
X = X.merge(travel_times, how='left', on='Date')
X.head(1)

,ride_id,travel_time,travel_from,max_capacity,Count,day,t,Date,Daily Mean Travel Time (Seconds)
0,1442,435,Migori,49,1.0,1,0,2017-10-17,532.0


In [74]:
# Create the model
model = CatBoostRegressor(iterations=100, 
                          depth=8, 
                          learning_rate=0.6, 
                          loss_function='MAE', verbose=False)

In [75]:
in_cols = ['travel_time', 'travel_from', 'max_capacity', 'day'] #'Daily Mean Travel Time (Seconds)' as an option

In [76]:
# Train model
tr = X.loc[X.t == 0]
model.fit(tr[in_cols], tr['Count'], cat_features=['travel_from', 'max_capacity', 'day'])

In [77]:
# Score model
print(mean_absolute_error(model.predict(tr[in_cols]), tr['Count']))

3.0238402948993057


In [78]:
# Make predictions and append to the sample submission data, and save as csv
te = X.loc[X.t == 1]
te[in_cols].head()
te = X.loc[X.t == 1]
sample['number_of_ticket'][5:] = model.predict(te[in_cols])[5:] # Ignore the warning
sample.to_csv('catboost_solution_2.79.csv', index=False)
sample.head(10)

,ride_id,number_of_ticket
0,4446,0.000000
1,13962,0.000000
2,5569,0.000000
3,1675,0.000000
4,5711,0.000000
5,2417,9.870357
6,15010,10.744758
7,1823,7.306642
8,15191,11.974640
9,14402,-12.318206


In [67]:
# Plot feature importance
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,travel_time,37.942220
1,travel_from,33.787444
2,day,19.550092
3,max_capacity,8.720244
